<a href="https://colab.research.google.com/github/anikaha/OCR_7/blob/main/cropBoundingBoxesToFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Import all library
import imageio
import pandas as pd
import numpy as np
import os
import glob
import cv2
import shutil

import imgaug as ia
ia.seed(1)
# imgaug uses matplotlib backend for displaying images
%matplotlib inline
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load CSV data ino df
df_data = pd.read_csv ('/content/drive/MyDrive/labels.csv')
df_data

In [ ]:
!unzip '/content/drive/MyDrive/OCR.zip' -d '/tmp/OCR'

In [7]:
data_path = '/tmp/OCR/ImageAndXML_Data/'

Show data and its bounding boxes

In [ ]:
grouped = df_data.groupby('filename')
# we can pull each group with get_group() using the filename

group_df = grouped.get_group('92214362.tif')
group_df = group_df.reset_index()
group_df = group_df.drop(['index'], axis=1)
group_df

In [ ]:
# get bounding boxes coordinates from grouped data frame and write into array        
bb_array1 = group_df.drop(['filename', 'width', 'height', 'id_big', 'xmin2', 'ymin2', 'xmax2', 'ymax2', 'conf', 'unicode'], axis=1).values
# display the array we've got
bb_array1 = np.unique(bb_array1, axis=0)
bb_array1

In [ ]:
# pass the array of bounding boxes coordinates to the imgaug library
im = imageio.imread(os.path.join(data_path,"92214362.tif"))
bbs = BoundingBoxesOnImage.from_xyxy_array(bb_array1, shape=im.shape)
# display the image and draw bounding boxes
backtorgb = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
a = bbs.draw_on_image(backtorgb)
ia.imshow(a)

In [ ]:
# get bounding boxes coordinates from grouped data frame and write into array        
bb_array = group_df.drop(['filename', 'width', 'height', 'id_big', 'xmin', 'ymin', 'xmax', 'ymax', 'conf', 'unicode'], axis=1).values
# display the array we've got
bb_array

In [ ]:
# pass the array of bounding boxes coordinates to the imgaug library
im = imageio.imread(os.path.join(data_path,"92214362.tif"))
bbs = BoundingBoxesOnImage.from_xyxy_array(bb_array, shape=im.shape)
# display the image and draw bounding boxes
backtorgb = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
a = bbs.draw_on_image(backtorgb)
ia.imshow(a)

Crop all files into small files based on its bounding box

In [ ]:
# all filenames
all_filenames = df_data['filename'].unique()
# crop all big box boxes into file
df_bigBox = df_data.drop(['xmin2', 'ymin2', 'xmax2', 'ymax2', 'conf', 'unicode'], axis=1)
df_bigBox = df_bigBox.drop_duplicates()
print(df_bigBox)
if not os.path.exists("/tmp/OCR/crop_big"):
  os.makedirs("/tmp/OCR/crop_big")
for index,row in df_bigBox.iterrows():
  (filename, width, height, id_big, xmin, ymin, xmax, ymax) = row
  im_path = os.path.join(data_path, filename)
  img = imageio.imread(im_path)
  img2 = img[ymin:ymax, xmin:xmax, ...]
  filename_o = os.path.join("/tmp/OCR/crop_big",(filename+"_"+id_big+".png"))
  imageio.imwrite(filename_o,img2)

In [66]:
!cp -R '/tmp/OCR/crop_big' '/content/drive/MyDrive/OCR'

In [ ]:
# crop all small box boxes into file
df_smallBox = df_data.drop(['id_big', 'xmin', 'ymin', 'xmax', 'ymax'], axis=1)
#print(df_smallBox)

if not os.path.exists("/tmp/OCR/crop_small"):
  os.makedirs("/tmp/OCR/crop_small")
for index,row in df_smallBox.iterrows():
  (filename, width, height, xmin, ymin, xmax, ymax, conf, unicod) = row
  im_path = os.path.join(data_path, filename)
  img = imageio.imread(im_path)
  img2 = img[ymin:ymax, xmin:xmax, ...]
  unicod = str(unicod).replace("/", "~")

  filename_o = os.path.join("/tmp/OCR/crop_small",(str(unicod)+"_"+str(conf)+".png"))
  print(filename_o)
  imageio.imwrite(filename_o,img2)


In [75]:
!cp -R '/tmp/OCR/crop_small' '/content/drive/MyDrive/OCR'